In [ ]:
import os
import sys
import importlib
from pathlib import Path

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [ ]:
from setup_module.imports import *


In [ ]:
# Function to remove stopwords
def remove_code(keywords_lst: list, keyword_clean_lst=None) -> list:

    if keyword_clean_lst is None:
        keyword_clean_lst = []

    for s in keywords_lst:
        lst = s.split()
        for i in lst:
            if len(i) <= 2:
                lst.remove(i)
            keyword_clean_lst.append(' '.join(lst))

    # map(lambda n: n * 2, [1, 2, 3, 4, 5])
    return [x for x in keyword_clean_lst if x != '']


In [ ]:
# Function to to check file exists and not empty
def is_non_zero_file(fpath):

    return (os.path.isfile(fpath) and os.path.getsize(fpath) > 0)


In [ ]:
# Function to validate path or file
def validate_path(file: str) -> str:

    if not os.path.isdir(file) and is_non_zero_file(file) is False:
        # file = input(f'No file found at {file}.\nPlease enter correct path.')
        try:
            print(f'File {file} not found.')
        except Exception as e:
            print(e.json())
    return file


In [ ]:
# Function to clean keyword list
def clean_and_translate_keyword_list(
    keywords_lst: list,
    translate_enabled = None
) -> list:

    if translate_enabled is None:
        translate_enabled = False
    elif translate_enabled is True:
        translator = Translator()

    assert all(isinstance(i, str) for i in keywords_lst), 'Keywords must be strings.'

    # Collect all and and comma containing keywords
    and_comma = [i for i in keywords_lst if (',' in i) or ('and' in i)]

    # Remove ands and commas and append to keywords
    if len(and_comma) > 0:
        for i in and_comma:
            for x in re.split('and|,', i.strip().lower()):
                keywords_lst.append(x.strip().lower())

        # Remove duplicates
        keywords_lst = list(set(keywords_lst) ^ set(and_comma))

    else:
        keywords_lst = list(set(keywords_lst))

    # # Remove codes
    keywords_lst = remove_code(keywords_lst)

    # Singularize and remove duplicates
    keywords_list = list(
        set(
            list(
                map(
                    lambda line: (Word(line.lower()).singularize()).lower(),
                    keywords_lst,
                )
            )
        )
    )

    # Remove all non-specific keywords
    for i in keywords_list:
        if 'other ' in i.lower() and i.lower() not in ['other business support', 'other service activities']:
            keywords_list.append(i.lower().split('other')[1])
            keywords_list.remove(i)
        if ' (excl.' in i.lower():
            keywords_list.append(i.lower().split(' (excl.')[0].lower())
            keywords_list.remove(i)
        if '_(excl' in i.lower():
            keywords_list.append(i.lower().split('_(excl')[0].lower())
            keywords_list.remove(i)
    for i in keywords_list:
        if ' (' in i.lower():
            keywords_list.append(i.lower().split(' (')[0].lower())
            keywords_list.remove(i)
        if "-Noon's" in i.lower():
            keywords_list.append(i.lower().split('-Noon')[0].lower())
            keywords_list.remove(i)
        if len(i) <= 2:
            keywords_list.remove(i)
    for i in keywords_list:
        for w_keyword, r_keyword in keyword_trans_dict.items():
            if str(i.lower()) == w_keyword.lower():
                keywords_list.remove(i)
                keywords_list.append(r_keyword)

    # Remove duplicates
    keywords_list = list(filter(None, list(set(keywords_list))))

    # Translate to Dutch
    if translate_enabled is True:
        for english_keyword in keywords_list:
            while True:
                try:
                    dutch_keyword = translator.translate(english_keyword).text
                except Exception as e:
                    time.sleep(0.3)
                    continue
                break
            keywords_list.append(dutch_keyword.lower())

        # Remove duplicates
        keywords_list = list(filter(None, list(set(keywords_list))))

    return list(
        filter(None, list(set([i.lower().strip() for i in keywords_list if i])))
    )


In [ ]:
# %% CBS Data request
def get_cbs_odata(
    sectors_file_path = None,
    table_url: str = None,
    table_id: str = None,
    addition_url: str = None,
    select: list=None,
):
    if sectors_file_path is None:
        sectors_file_path: str = validate_path(f'{code_dir}/scraped_data/CBS/Found Data/')
    if table_url is None:
        table_url='https://opendata.cbs.nl/ODataAPI/OData/'
    if table_id is None:
        table_id='81434ENG'
    if addition_url is None:
        addition_url='/UntypedDataSet'
    if select is None:
        select=['SexOfEmployee', 'TypeOfEmploymentContract', 'OtherCharacteristicsEmployee', 'IndustryClassBranchSIC2008', 'Periods', 'Jobs_1']
    # data: https://opendata.cbs.nl/#/CBS/en/dataset/81434ENG/table?ts=1663627369191
    # instruction: https://data.overheid.nl/dataset/410-bevolking-op-eerste-van-de-maand--geslacht--leeftijd--migratieachtergrond
    # github: https://github.com/statistiekcbs/CBS-Open-Data-v4

    tables = cbsodata.get_table_list()
    for table in tables:
        if table['Identifier'] == table_id:
            data_info = table
    info = cbsodata.get_info(table_id)
    diffs = list(set(info.keys()) - set(data_info.keys()))
    for i in diffs:
        data_info[i] = info[i]

    with open(f'{sectors_file_path}cbs_data_info.json', 'w', encoding='utf8') as f:
        json.dump(data_info, f)

    dimensions = defaultdict(dict)
    for sel in select:
        if sel != 'Jobs_1':
            meta_data = pd.DataFrame(cbsodata.get_meta('81434ENG', sel))
        if sel == 'TypeOfEmploymentContract':
            meta_data = meta_data.loc[~meta_data['Title'].str.contains('Type of employment contract:')]
        if sel == 'OtherCharacteristicsEmployee':
            meta_data = meta_data.loc[~meta_data['Key'].str.contains('NAT')]
        if sel == 'Periods':
            meta_data = meta_data.loc[meta_data['Title'].astype(str) == '2020']

        for title, key in zip(meta_data['Title'].tolist(), meta_data['Key'].tolist()):
            if sel != 'Jobs_1':
                dimensions[sel][title] = key
    with open(f'{sectors_file_path}cbs_data_dimensions.json', 'w', encoding='utf8') as f:
        json.dump(dimensions, f)

    while True:
        try:
            data = pd.DataFrame(cbsodata.get_data(table_id, select=select))
            break
        except ConnectionError:
            time.sleep(5)

    data = data.loc[~data['TypeOfEmploymentContract'].str.contains('Type of employment contract:') & ~data['OtherCharacteristicsEmployee'].str.contains('Nationality:') & data['Periods'].str.contains('2020')]

    data.to_csv(f'{sectors_file_path}Sectors Tables/FINAL/Gender x Age_CBS_DATA_from_code.csv')

    # target_url = table_url + table_id + addition_url

    # data = pd.DataFrame()
    # while target_url:
    #     r = requests.get(target_url).json()
    #     data = data.append(pd.DataFrame(r['value']))

    #     if '@odata.nextLink' in r:
    #         target_url = r['@odata.nextLink']
    #     else:
    #         target_url = None

    return data


In [ ]:
# %% Function to get sbi_sectors_dict
def get_sbi_sectors_list(
    save_enabled=None,
    parent_dir=None,
    ):

    if save_enabled is None:
        save_enabled=True
    if parent_dir is None:
        parent_dir=validate_path(f'{code_dir}/scraped_data/CBS/')

    sib_5_loc = validate_path(f'{parent_dir}Found Data/SBI_ALL_NACE_REV2.csv')
    data_save_dir = validate_path(f'{parent_dir}Data/')

    df_sbi_sectors = pd.read_csv(sib_5_loc, delimiter=',')
    df_sbi_sectors.columns = df_sbi_sectors.columns.str.strip()
    df_sbi_sectors = df_sbi_sectors.rename(columns = {'Description': 'Old_Sector_Name'})
    df_sbi_sectors = df_sbi_sectors.dropna(subset=['Old_Sector_Name', 'Code'])
    df_sbi_sectors['Old_Sector_Name'] = df_sbi_sectors['Old_Sector_Name'].progress_apply(lambda x: x.lower().strip())
    df_sbi_sectors = df_sbi_sectors.loc[df_sbi_sectors['Level'] == 1]
    df_sbi_sectors = df_sbi_sectors.drop(columns=['Level', 'Parent', 'This item includes', 'This item also includes', 'Rulings', 'This item excludes', 'Reference to ISIC Rev. 4'])

    df_sectors_all = pd.read_pickle(f'{data_save_dir}Sectors Output from script.pkl')[[('SBI Sector Titles'), ('Gender'), ('Age')]].droplevel('Categories', axis='columns')[[('SBI Sector Titles', 'Code'), ('SBI Sector Titles', 'Sector Name'), ('SBI Sector Titles', 'Keywords'), ('Gender', 'Dominant Category'), ('Age', 'Dominant Category')]].droplevel('Variables', axis='columns')
    df_sectors_all.columns = ['Code', 'Sector Name', 'Keywords', 'Gender Dominant Category', 'Age Dominant Category']
    df_sbi_sectors = df_sbi_sectors.merge(df_sectors_all, how='inner', on='Code')
    df_sbi_sectors = df_sbi_sectors.rename(columns = {'Sector Name': 'Sector_Name', 'Keywords': 'Used_Sector_Keywords', 'Gender Dominant Category': 'Gender_Dominant_Category', 'Age Dominant Category': 'Age_Dominant_Category'})
    df_sbi_sectors['Sector_Name'] = df_sbi_sectors['Sector_Name'].progress_apply(lambda x: x.strip().lower() if isinstance(x, str) else np.nan)
    df_sbi_sectors['Used_Sector_Keywords'] = df_sbi_sectors['Used_Sector_Keywords'].progress_apply(lambda x: clean_and_translate_keyword_list(x) if isinstance(x, list) else np.nan)
    df_sbi_sectors = df_sbi_sectors.set_index(df_sbi_sectors['Code'])

    df_sbi_sectors.to_csv(f'{data_save_dir}SBI-5_Sectors.csv', index=True)
    df_sbi_sectors.to_excel(f'{data_save_dir}SBI-5_Sectors.xlsx', index=True)
    df_sbi_sectors.to_pickle(f'{data_save_dir}SBI-5_Sectors.pkl')

    sbi_english_keyword_list = [i for index, row in df_sbi_sectors['Used_Sector_Keywords'].items() if isinstance(row, list) for i in row]
    sbi_english_keyword_list = clean_and_translate_keyword_list(sbi_english_keyword_list)

    sbi_english_keyword_dict = df_sbi_sectors['Used_Sector_Keywords'].to_dict()
    sbi_sectors_dict = df_sbi_sectors.to_dict('index')
    sbi_sectors_dict_full = {}
    sbi_sectors_dom_gen = {}
    sbi_sectors_dom_age = {}
    sbi_sectors_keywords_gen_dom = defaultdict(list)
    sbi_sectors_keywords_age_dom = defaultdict(list)
    sbi_sectors_keywords_full_dom = defaultdict(list)
    for index, row in df_sbi_sectors.iterrows():
        sbi_sectors_dict_full[row['Sector_Name']] = row['Used_Sector_Keywords']
        sbi_sectors_dom_gen[row['Sector_Name']] = row['Gender_Dominant_Category']
        sbi_sectors_dom_age[row['Sector_Name']] = row['Age_Dominant_Category']
    for cat_keywords in df_sbi_sectors[['Gender_Dominant_Category', 'Used_Sector_Keywords']].to_dict(orient='split')['data']:
        sbi_sectors_keywords_gen_dom[cat_keywords[0]].extend(cat_keywords[1])
    for cat_keywords in df_sbi_sectors[['Age_Dominant_Category', 'Used_Sector_Keywords']].to_dict(orient='split')['data']:
        sbi_sectors_keywords_age_dom[cat_keywords[0]].extend(cat_keywords[1])
    for d in (sbi_sectors_keywords_gen_dom, sbi_sectors_keywords_age_dom):
        sbi_sectors_keywords_full_dom |= d

    if save_enabled is True:
        with open(f'{data_save_dir}sbi_english_keyword_list.txt', 'w', encoding='utf8') as f:
            for i in sbi_english_keyword_list:
                f.write(f'{i.lower()}\n')
        with open(f'{data_save_dir}sbi_english_keyword_dict.json', 'w', encoding='utf8') as f:
            json.dump(sbi_english_keyword_dict, f)
        with open(f'{data_save_dir}sbi_sectors_dict.json', 'w', encoding='utf8') as f:
            json.dump(sbi_sectors_dict, f)

        with open(f'{data_save_dir}sbi_sectors_keywords_gen_dom.json', 'w', encoding='utf8') as f:
            json.dump(sbi_sectors_keywords_gen_dom, f)
        with open(f'{data_save_dir}sbi_sectors_keywords_age_dom.json', 'w', encoding='utf8') as f:
            json.dump(sbi_sectors_keywords_age_dom, f)
        with open(f'{data_save_dir}sbi_sectors_keywords_full_dom.json.json', 'w', encoding='utf8') as f:
            json.dump(sbi_sectors_keywords_full_dom, f)

    return (
        sbi_english_keyword_list,
        sbi_english_keyword_dict,
        sbi_sectors_dict,
        sbi_sectors_dict_full,
        sbi_sectors_dom_gen,
        sbi_sectors_dom_age,
        sbi_sectors_keywords_gen_dom,
        sbi_sectors_keywords_age_dom,
        sbi_sectors_keywords_full_dom
    )


In [ ]:
# Function to rearrgane gender and age columns
def get_only_df(df_sectors, col_name, opp_col_name):
    df_only = df_sectors.pivot_table(values='n', index=['Code', 'Sector Name', opp_col_name], columns=[col_name], aggfunc='sum')
    df_only = df_only.reset_index()
    df_only = df_only.loc[df_only[opp_col_name] == 'Total']
    df_only = df_only.drop(columns=[opp_col_name, 'Total'])
    df_only.name = col_name

    return df_only


In [ ]:
def save_sector_excel(
    df_sectors_all,
    tables_file_path,
    sheet_name=None,
    excel_file_name=None,
    age_limit: int=None,
    age_ratio: int=None,
    gender_ratio: int=None,
):
    if sheet_name is None:
        sheet_name = 'All'
    if excel_file_name is None:
        excel_file_name = 'Sectors Output from script.xlsx'
    if age_limit is None:
        age_limit = 45
    if age_ratio is None:
        age_ratio = 10
    if gender_ratio is None:
        gender_ratio = 20

    writer = pd.ExcelWriter(f'{tables_file_path}{excel_file_name}', engine='xlsxwriter')
    df_sectors_all.to_excel(writer, sheet_name=sheet_name, merge_cells = True, startrow = 3)
    workbook  = writer.book
    worksheet = writer.sheets[sheet_name]
    worksheet.set_row(6, None, None, {'hidden': True})
    worksheet.set_column(0, 0, None, None, {'hidden': True})
    # Title
    worksheet.merge_range(0, 0, 0, df_sectors_all.shape[1], 'Table', workbook.add_format({'bold': True, 'font_name': 'Times New Roman', 'font_size': 12, 'font_color': 'black', 'align': 'left'}))
    worksheet.merge_range(1, 0, 1, df_sectors_all.shape[1], 'Sectoral Gender and Age Composition and Segregation, Keywords, Counts, and Percentages', workbook.add_format({'italic': True, 'font_name': 'Times New Roman', 'font_size': 12, 'font_color': 'black', 'align': 'left'}))
    worksheet.merge_range(2, 0, 2, df_sectors_all.shape[1], 'Jobs Count per Sector (x 1000)', workbook.add_format({'bold': False, 'font_name': 'Times New Roman', 'font_size': 12, 'font_color': 'black', 'align': 'center', 'top': True, 'bottom': True}))
    # Format column headers
    for col_num, value in enumerate(df_sectors_all.columns.values):
        for i in range(3):
            worksheet.write(3 + i, col_num + 1, value[i], workbook.add_format({'bold': False, 'font_name': 'Times New Roman', 'font_size': 12, 'font_color': 'black', 'align': 'center', 'top': True, 'bottom': True}))
            if value[i] == 'n':
                worksheet.set_column(col_num + 1, col_num + 1, 5.5)
            elif value[i] == 'Code':
                worksheet.set_column(col_num + 1, col_num + 1, 4.5)
            elif value[i] == 'Sector Name':
                worksheet.set_column(col_num + 1, col_num + 1, 28.5)
            elif value[i] == 'Keywords':
                worksheet.set_column(col_num + 1, col_num + 1, 30)
            elif value[i] == 'Keywords Count':
                worksheet.set_column(col_num + 1, col_num + 1, 13.5)
            elif value[i] == '% per Sector':
                worksheet.set_column(col_num + 1, col_num + 1, 12)
            elif value[i] == '% per Social Category':
                worksheet.set_column(col_num + 1, col_num + 1, 19.5)
            elif value[i] == '% per Workforce':
                worksheet.set_column(col_num + 1, col_num + 1, 15.5)
            elif value[i] == 'Dominant Category':
                worksheet.set_column(col_num + 1, col_num + 1, 24.5)
            elif value[i] == '% Sector per Workforce':
                worksheet.set_column(col_num + 1, col_num + 1, 21.5)

    # Borders
    perc = [col_num for col_num, value in enumerate(df_sectors_all.columns.values) if '%' in value[-1]]
    num = [col_num for col_num, value in enumerate(df_sectors_all.columns.values) if 'n' in value[-1]]
    word = [col_num for col_num, value in enumerate(df_sectors_all.columns.values) if value[-1] in ['Code', 'Sector Name', 'Dominant Category']]
    keyword = [col_num for col_num, value in enumerate(df_sectors_all.columns.values) if 'Keywords' in value[-1]]

    row_idx, col_idx = df_sectors_all.shape
    for c, r in itertools.product(range(col_idx), range(row_idx)):
        if c in perc:
            formats = {'num_format': '0.00', 'font_name': 'Times New Roman', 'font_size': 12}
            if r == row_idx-1:
                formats |= {'top': True, 'bottom': True}
        elif c in num:
            formats = {'num_format': '0', 'font_name': 'Times New Roman', 'font_size': 12}
            if r == row_idx-1:
                formats |= {'top': True, 'bottom': True}
        elif c in word:
            formats = {'font_name': 'Times New Roman', 'font_size': 12}
            if r == row_idx-1:
                formats |= {'top': True, 'bottom': True}
        elif c in keyword:
            formats = {'font_name': 'Times New Roman', 'font_size': 12, 'align': 'left'}
            if r == row_idx-1:
                formats |= {'top': True, 'bottom': True}
        try:
            worksheet.write(r + 7, c + 1, df_sectors_all.iloc[r, c], workbook.add_format(formats))
        except TypeError:
            value = (
                str(df_sectors_all.iloc[r, c])
                if isinstance(df_sectors_all.iloc[r, c], list)
                else ''
            )
            worksheet.write(r + 7, c + 1, value, workbook.add_format(formats))

    worksheet.merge_range(len(df_sectors_all)+7, 0, len(df_sectors_all)+7, df_sectors_all.shape[1], 'Note.', workbook.add_format({'italic': True, 'font_name': 'Times New Roman', 'font_size': 10, 'font_color': 'black', 'align': 'left'}))
    worksheet.merge_range(len(df_sectors_all)+8, 0, len(df_sectors_all)+8, df_sectors_all.shape[1], f'Threshold for gender = {df_sectors_all.loc[df_sectors_all.index[-1], ("Gender", "Female", "% per Workforce")]:.2f}% ± 20%', workbook.add_format({'italic': True, 'font_name': 'Times New Roman', 'font_size': 10, 'font_color': 'black', 'align': 'left'}))
    worksheet.merge_range(len(df_sectors_all)+9, 0, len(df_sectors_all)+9, df_sectors_all.shape[1], f'Threshold for age = {df_sectors_all.loc[df_sectors_all.index[-1], ("Age", f"Older (>= {age_limit} years)", "% per Workforce")]:.2f}% ± 10%', workbook.add_format({'italic': True, 'font_name': 'Times New Roman', 'font_size': 10, 'font_color': 'black', 'align': 'left'}))
    worksheet.merge_range(len(df_sectors_all)+10, 0, len(df_sectors_all)+10, df_sectors_all.shape[1], 'Source: Centraal Bureau voor de Statistiek (CBS)', workbook.add_format({'italic': True, 'font_name': 'Times New Roman', 'font_size': 8, 'font_color': 'black', 'align': 'left'}))

    writer.close()


In [ ]:
# Function to get sector df from cbs
def get_sector_df_from_cbs(
    save_enabled: bool = True,
    parent_dir=validate_path(f'{code_dir}/scraped_data/CBS/'),
    cols = ['Industry class / branch (SIC2008)', 'Sex of employee', 'Other characteristics employee', 'Employment/Jobs (x 1 000)'],
    get_cbs_odata_enabled=False,
    age_limit: int = 45,
    age_ratio: int = 10,
    gender_ratio: int = 20,
):

    sectors_file_path: str = validate_path(f'{parent_dir}Found Data/')
    data_save_dir1: str = validate_path(f'{parent_dir}Data/')
    data_save_dir2: str = validate_path(f'{code_dir}/data/output tables/')

    # with open(f'{code_dir}/data/content analysis + ids + sectors/sbi_sectors_dict.json', 'r', encoding='utf8') as f:
    #     sbi_sectors_dict = json.load(f)
    sbi_english_keyword_list, sbi_english_keyword_dict, sbi_sectors_dict, sbi_sectors_dict_full, sbi_sectors_dom_gen, sbi_sectors_dom_age, sbi_sectors_keywords_gen_dom, sbi_sectors_keywords_age_dom, sbi_sectors_keywords_full_dom = get_sbi_sectors_list()

    if get_cbs_odata_enabled is True:
        select = ['SexOfEmployee', 'TypeOfEmploymentContract', 'OtherCharacteristicsEmployee', 'IndustryClassBranchSIC2008', 'Periods', 'Jobs_1']
        odata_colnames_normalized = {'IndustryClassBranchSIC2008': 'Industry class / branch (SIC2008)', 'SexOfEmployee': 'Sex of employee', 'OtherCharacteristicsEmployee': 'Other characteristics employee', 'Jobs_1': 'Employment/Jobs (x 1 000)'}
        df_sectors = get_cbs_odata()
        df_sectors = df_sectors.rename(columns=odata_colnames_normalized)
    elif get_cbs_odata_enabled is False:
        # print(f'Error getting data from CBS Statline OData. Using the following file:\n{sectors_file_path}Sectors Tables/FINAL/Gender x Age_CBS_DATA.csv')
        # Read, clean, create code variable
        try:
            df_sectors = pd.read_csv(f'{sectors_file_path}Sectors Tables/FINAL/Gender x Age_CBS_DATA.csv', delimiter=';')
        except Exception:
            df_sectors = pd.read_csv(f'{sectors_file_path}Sectors Tables/FINAL/Gender x Age_CBS_DATA_from_code.csv', delimiter=';')


    df_sectors = df_sectors[cols]
    df_sectors = df_sectors.rename({'Sex of employee': 'Gender', 'Other characteristics employee': 'Age Range (in years)', 'Industry class / branch (SIC2008)': 'Sector Name', 'Employment/Jobs (x 1 000)': 'n'}, axis = 1)
    df_sectors.insert(0, 'Code', df_sectors['Sector Name'].progress_apply(lambda row: row[0]))
    df_sectors['Sector Name'] = df_sectors['Sector Name'].progress_apply(lambda row: row[2:].strip() if '-' not in row else row[3:].strip())

    # Categorize by age label
    all_age = df_sectors['Age Range (in years)'].unique().tolist()[1:]
    for i, word in enumerate(all_age):
        if word.startswith(str(age_limit)):
            young = all_age[:i]
            old = all_age[i:]
    conditions = [
        (df_sectors['Age Range (in years)'].isin(old)),
        (df_sectors['Age Range (in years)'].isin(young))
    ]
    choices = [f'Older (>= {age_limit} years)', f'Younger (< {age_limit} years)']
    age_cat = np.select(conditions, choices, default='Total')
    df_sectors.insert(3, 'Age', age_cat)
    choices.append('Total')

    # Change gender label
    df_sectors['Gender'] = df_sectors['Gender'].replace({'Sex: Female': 'Female', 'Sex: Male': 'Male'})

    # Rearrgane columns
    # Gender
    df_gender_only = get_only_df(df_sectors, 'Gender', 'Age')

    # Age
    df_age_only = get_only_df(df_sectors, 'Age', 'Gender')

    # Total
    df_total_only = df_sectors.pivot_table(values='n', index=['Code', 'Sector Name', 'Gender', 'Age'], aggfunc='sum')
    df_total_only = df_total_only.reset_index()
    df_total_only = df_total_only.loc[(df_total_only['Gender'] == 'Total') & (df_total_only['Age'] == 'Total')]
    df_total_only = df_total_only.drop(columns=['Gender', 'Age'])
    df_total_only = df_total_only.rename(columns={'n': 'Total Workforce'})
    df_total_only.name = 'Total'

    # Merge all
    df_sectors_all = pd.merge(pd.merge(df_gender_only, df_age_only, how='outer'), df_total_only, how='outer')

    # Take out "All economic activities" row
    au = df_sectors_all.loc[df_sectors_all['Sector Name'] == 'All economic activities']
    au.loc[au['Code'] != 'A-U', 'Code'] = 'A-U'
    df_sectors_all = df_sectors_all[df_sectors_all['Sector Name'] != 'All economic activities']
    df_sectors_all = df_sectors_all.groupby(['Code'], as_index=True).agg({'Sector Name': 'first', **dict.fromkeys(df_sectors_all.loc[:, ~df_sectors_all.columns.isin(['Code', 'Sector Name'])].columns.to_list(), 'sum')})
    df_sectors_all = df_sectors_all.reset_index()

    # Add keywords
    df_sectors_all.insert(2, 'Keywords', df_sectors_all['Code'].progress_apply(lambda row: sbi_sectors_dict[row]['Used_Sector_Keywords'] if row in sbi_sectors_dict and isinstance(row, str) else np.nan))
    df_sectors_all['Keywords'] = df_sectors_all['Keywords'].progress_apply(lambda row: clean_and_translate_keyword_list(row) if isinstance(row, list) else np.nan)
    df_sectors_all.insert(
        3,
        'Keywords Count',
        df_sectors_all['Keywords'].progress_apply(
            lambda row: len(row) if isinstance(row, list) else np.nan
        ),
    )

    # Add totals in bottom row
    df_sectors_all.loc[df_sectors_all[df_sectors_all['Sector Name'] == 'Other service activities'].index.values.astype(int)[0]+1, 'Sector Name'] = 'Total (excluding A-U)'
    df_sectors_all.iloc[df_sectors_all[df_sectors_all['Sector Name'] == 'Total (excluding A-U)'].index.values.astype(int)[0], ~df_sectors_all.columns.isin(['Code', 'Sector Name', 'Keywords'])] = df_sectors_all.sum(numeric_only=True)
    df_sectors_all.columns = pd.MultiIndex.from_tuples([('Industry class / branch (SIC2008)', 'Code'), ('Industry class / branch (SIC2008)', 'Sector Name'), ('Industry class / branch (SIC2008)', 'Keywords'), ('Industry class / branch (SIC2008)', 'Keywords Count'), ('Female', 'n'), ('Male', 'n'), (f'Older (>= {age_limit} years)', 'n'), (f'Younger (< {age_limit} years)', 'n'), ('Total Workforce', 'n')], names = ['Social category', 'Counts'])

    # Make percentages
    for index, row in df_sectors_all.items():
        if ('Total' not in index[0]) and ('%' not in index[1]) and ('n' in index[1]) and (not isinstance(row[0], str)) and (not math.isnan(row[0])):
            df_sectors_all[(index[0], '% per Sector')] = row/df_sectors_all[('Total Workforce', 'n')]*100
            df_sectors_all[(index[0], '% per Social Category')] = row/df_sectors_all.loc[df_sectors_all[df_sectors_all[('Industry class / branch (SIC2008)', 'Sector Name')] == 'Total (excluding A-U)'].index.values.astype(int)[0], index]*100
            df_sectors_all[(index[0], '% per Workforce')] = row/df_sectors_all.loc[df_sectors_all[df_sectors_all[('Industry class / branch (SIC2008)', 'Sector Name')] == 'Total (excluding A-U)'].index.values.astype(int)[0], ('Total Workforce', 'n')]*100
        if ('Total' in index[0]):
            df_sectors_all[(index[0], '% Sector per Workforce')] = row/df_sectors_all.loc[df_sectors_all[df_sectors_all[('Industry class / branch (SIC2008)', 'Sector Name')] == 'Total (excluding A-U)'].index.values.astype(int)[0], ('Total Workforce', 'n')]*100

    # Set cut-off
    # Gender
    total_female = df_sectors_all.loc[df_sectors_all[df_sectors_all[('Industry class / branch (SIC2008)', 'Sector Name')] == 'Total (excluding A-U)'].index.values.astype(int)[0], ('Female', '% per Workforce')]
    female_dominated = total_female + (gender_ratio / 100)
    df_sectors_all.loc[df_sectors_all[('Female', '% per Sector')] >= female_dominated, ('Sectoral Gender Segregation', 'Dominant Category')] = 'Female'
    male_dominated = total_female - (gender_ratio / 100)
    df_sectors_all.loc[df_sectors_all[('Female', '% per Sector')] <= male_dominated, ('Sectoral Gender Segregation', 'Dominant Category')] = 'Male'
    df_sectors_all.loc[(df_sectors_all[('Female', '% per Sector')] > male_dominated) & (df_sectors_all[('Female', '% per Sector')] < female_dominated) & (df_sectors_all[('Industry class / branch (SIC2008)', 'Sector Name')].astype(str) != 'Total (excluding A-U)'), ('Sectoral Gender Segregation', 'Dominant Category')] = 'Mixed Gender'
    # Age
    total_old = df_sectors_all.loc[df_sectors_all[df_sectors_all[('Industry class / branch (SIC2008)', 'Sector Name')] == 'Total (excluding A-U)'].index.values.astype(int)[0], (f'Older (>= {age_limit} years)', '% per Workforce')]
    old_dominated = total_old + age_ratio / 100
    df_sectors_all.loc[df_sectors_all[(f'Older (>= {age_limit} years)', '% per Sector')] >= old_dominated, ('Sectoral Age Segregation', 'Dominant Category')] = 'Older'
    young_dominated = total_old - age_ratio / 100
    df_sectors_all.loc[df_sectors_all[(f'Older (>= {age_limit} years)', '% per Sector')] <= young_dominated, ('Sectoral Age Segregation', 'Dominant Category')] = 'Younger'
    df_sectors_all.loc[(df_sectors_all[(f'Older (>= {age_limit} years)', '% per Sector')] < old_dominated) & (df_sectors_all[(f'Older (>= {age_limit} years)', '% per Sector')] > young_dominated) & (df_sectors_all[('Industry class / branch (SIC2008)', 'Sector Name')].astype(str) != 'Total (excluding A-U)'), ('Sectoral Age Segregation', 'Dominant Category')] = 'Mixed Age'

    # Add AU and other rows
    au.insert(2, 'Keywords', np.nan)
    au.insert(3, 'Keywords Count', np.nan)
    au[['Sectoral Gender Segregation', 'Sectoral Age Segregation']] = np.nan
    au.columns = pd.MultiIndex.from_tuples([col for col in df_sectors_all.columns if '%' not in col[1]])
    df_sectors_all = pd.concat([au, df_sectors_all], ignore_index=True)

    # Arrange columns
    df_sectors_all = df_sectors_all.reindex(columns=df_sectors_all.columns.reindex(['Industry class / branch (SIC2008)', 'Female', 'Male', 'Sectoral Gender Segregation', f'Older (>= {age_limit} years)', f'Younger (< {age_limit} years)', 'Sectoral Age Segregation', 'Total Workforce'], level=0)[0])
    df_sectors_all = df_sectors_all.reindex(columns=df_sectors_all.columns.reindex(['Code', 'Sector Name', 'Keywords', 'Keywords Count', 'n', '% per Sector', '% per Social Category', '% per Workforce', '% Sector per Workforce', 'Dominant Category'], level=1)[0])

    level1_cols_tuple = []
    for col in df_sectors_all.columns:
        if ('SIC2008' in col[0]):
            level1_cols_tuple.append(('SBI Sector Titles', *col))
        elif (re.search(r'[Mm]ale', col[0])) or ('Gender' in col[0]):
            level1_cols_tuple.append(('Gender', *col))
        elif ('45' in col[0]) or ('Age' in col[0]):
            level1_cols_tuple.append(('Age', *col))
        elif ('Total' in col[0]):
            level1_cols_tuple.append(('Total Workforce', *col))

    df_sectors_all.columns = pd.MultiIndex.from_tuples(level1_cols_tuple, names=['Variables', 'Categories', 'Counts'])

    if save_enabled:
        for data_save_dir in [data_save_dir1, data_save_dir2]:
            df_sectors_all.to_csv(f'{data_save_dir}Sectors Output from script.csv', index=False)
            df_sectors_all.to_pickle(f'{data_save_dir}Sectors Output from script.pkl')
            with pd.option_context('max_colwidth', 10000000000):
                df_sectors_all.to_latex(f'{data_save_dir}Sectors Output from script.tex', index=False, longtable=True, escape=True, multicolumn=True, multicolumn_format='c', position='H', caption='Sectoral Gender and Age Composition and Segregation, Keywords, Counts, and Percentages', label='Jobs Count per Sector (x 1000)')
            df_sectors_all.to_markdown(f'{data_save_dir}Sectors Output from script.md', index=True)
            save_sector_excel(df_sectors_all, data_save_dir)

    return df_sectors_all


In [ ]:
# Get the list of keywords for the SBI sectors
(
    sbi_english_keyword_list,
    sbi_english_keyword_dict,
    sbi_sectors_dict,
    sbi_sectors_dict_full,
    sbi_sectors_dom_gen,
    sbi_sectors_dom_age,
    sbi_sectors_keywords_gen_dom,
    sbi_sectors_keywords_age_dom,
    sbi_sectors_keywords_full_dom
) = get_sbi_sectors_list()


In [ ]:
sbi_sectors_dict['A']

In [ ]:
# Get the table for Sector Composition + used keyowrds + classification of dominant caterory
df_sectors_all = get_sector_df_from_cbs()

In [ ]:
df_sectors_all

In [ ]:
df_sectors_all.columns

In [ ]:
df_sectors_all[('Total Workforce', 'Total Workforce', '% Sector per Workforce')]